In [52]:
# This script exports some data from the pickle file into a csv
# output/dtm_processed_output.p -> year_doc_topic.csv

In [48]:
import numpy as np
import pandas as pd
import pickle

In [49]:
alldata_new = pickle.load(open('output/dtm_processed_output.p', 'rb'))

In [50]:
term_topic = alldata_new['term_topic']
doc_topic = alldata_new['doc_topic']
docnames = alldata_new['docnames']
terms = alldata_new['terms']
docs_per_year = alldata_new['docs_per_year']

In [51]:
# structure of term_topic
year = 0
topic = 0
term = 0
prob = term_topic[year,topic,term]
prob

0.0022569877025296011

In [52]:
topics = range(term_topic.shape[1])

In [53]:
# topic label = most frequent word given topic

def topic_label(topic, term_topic, terms):
    term_freqs = np.sum(term_topic[:,topic,:], axis = 0)
    max_term = np.argsort(-term_freqs)[0]
    return(terms[max_term])

topic_labels = [topic_label(topic, term_topic, terms) for topic in topics]

topic_labels

['sequence',
 'face',
 'reasoning',
 'text',
 'color',
 'decision',
 'sentence',
 'causal',
 'concept',
 'child',
 'spatial',
 'error',
 'category',
 'english',
 'student',
 'network',
 'speaker',
 'probability',
 'agent',
 'visual']

In [54]:
','.join(['a','b'])

'a,b'

In [55]:
# topic description = 5 most frequent words given topic

def topic_label(topic, term_topic, terms):
    term_freqs = np.sum(term_topic[:,topic,:], axis = 0)
    top_terms = np.argsort(-term_freqs)[0:6]
    return(','.join(terms[top_terms]))

topic_labels = [topic_label(topic, term_topic, terms) for topic in topics]

topic_labels

TypeError: only integer scalar arrays can be converted to a scalar index

In [60]:

### GOAL:  p(topic|year)

# create year_doc, that shows which document belongs to which year

# use doc_topic, where doc_topic[doc,topic] has p(topic|doc)

# create year_doc_topic.csv, 
# which has the probability of a given topic in a given document in a given year


In [56]:
## year_doc:
## which document belongs to which year

year_doc_count=docs_per_year.copy()

x = []
year = 0
doc_id = 0
while year < len(year_doc_count):
    if year_doc_count[year] == 0:
        year += 1
    else:
        year_doc_count[year] -= 1
        x.append({'year': year, 'doc_id': doc_id, 'doc_name': docnames[doc_id]})
        doc_id += 1
year_doc = pd.DataFrame(x)
# year_doc = year_doc[['year','doc_id','doc_name']]
year_doc.head()

,doc_id,doc_name,year
0,0,Using Referential Communication to Study Menta...,0
1,1,Ungrammatical Influences Evidence for Dynamica...,0
2,2,Point-Light Displays Illuminate the Abstract N...,0
3,3,Are Retrievals from Long-Term Memory Interrupt...,0
4,4,The Representational Effect in Complex Systems...,0


In [57]:
## create year_doc_topic

# replicate year_doc for each topic
x = []
for topic in topics:
    dd = year_doc.copy()
    dd['topic'] = topic
    dd['topic_label'] = topic_labels[topic]
    x.append(dd)
year_doc_topic = pd.concat(x)


In [58]:
# use doc_topic, where doc_topic[doc,topic] has p(topic|doc)
doc_topic[0,0]

3.0978935057764542e-05

In [59]:
# given year and topic, go through docs from that year and get p(topic|doc)
def get_prob(row):
    return(doc_topic[row.doc_id, row.topic])

year_doc_topic['prob'] = year_doc_topic.apply(get_prob, axis=1)  # apply get_prob for each row (axis=1)

In [60]:
# save csv
year_doc_topic.to_csv("year_doc_topic.csv", index=False)